In [1]:
import os, re,json, imp, unittest
import pandas as pd

print('First Directory at :', os.getcwd()[2:])

#Load Setting File
with open('setting.txt','r') as f:
    setting = json.load(f)
print('Load Setting OK')    

os.chdir(re.sub(os.getcwd().split('\\')[-1]+'$','',os.getcwd()))
print('Change Directory into :', os.getcwd()[2:])

First Directory at : \01_DEV_Function\Shared_Function\tests
Load Setting OK
Change Directory into : \01_DEV_Function\Shared_Function


In [2]:
local_library = True
#local_library = False

if local_library : 
    da_tran_SQL = imp.load_source('da_tran_SQL', 'py_topping/data_connection/database.py').da_tran_SQL
else :
    from py_topping.data_connection.database import da_tran_SQL
    
def help_test(df_in1, df_in2) :
    result = list()
    expect = list()
    for i in df_in1.columns :
        result_list = list(df_in1[i])
        expect_list = list(df_in2[i])
        result_list.sort()
        expect_list.sort()
        result.append(result_list)
        expect.append(expect_list)
    return result, expect

# MS SQL

## Default MSSQL

In [3]:
test_mssql = setting['MSSQL']

class SQL_Test_class(unittest.TestCase):
    def setUp(self):
        self.sql = da_tran_SQL(sql_type = test_mssql['type'],
                                host_name = test_mssql['host'],
                                database_name = test_mssql['database'],
                                user = test_mssql['user'],
                                password = test_mssql['password'])
        self.df1 = pd.DataFrame({'col1' : [1,2,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        self.df2 = pd.DataFrame({'col1' : [4,5,6,7,8] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        self.table_name = 'unit_test_git'
        self.df1.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')

    def test_sp(self):
        sql_q = """ CREATE OR ALTER PROCEDURE unit_test_git_SP (@PARAM1 AS VARCHAR(100))
                    AS
                    BEGIN
                    SELECT * FROM {} WHERE col1 = @PARAM1 
                    END""".format(self.table_name)
        self.sql.engine.execute(sql_q)
        df_read = self.sql.read('unit_test_git_SP', SP = True , param = {'@PARAM1' : 1})
        expect_df = self.df1[self.df1['col1'] == 1]
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_read_sql(self):
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_whole(self):
        self.sql.dump_whole(self.df2, self.table_name)
        df_read = self.sql.read(self.table_name)
        expect_df = self.df2
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    
    def test_replace1(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,3,3,4,4,5] , 'col3' : [1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace1_str(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = 'col1')
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,3,3,4,4,5] , 'col3' : [1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace2(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,4,5,6,7,8] , 'col2' : [1,1,2,2,3,3,4,4,5] , 'col3' : [1,1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace3(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2','col3'])
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1.append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace_logic(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'],
                             math_logic = {'col1' : {'logic' : '>', 'value' : 1}})
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1[self.df1['col1'] <= 1].append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace_logic_date(self) :
        df_date = pd.DataFrame({'col1' : [1,2,3,4,5], 
                                'date' : pd.date_range('2020-10-10','2020-10-14'), 'col2' :[1,1,1,1,1]})
        
        self.sql.dump_whole(df_date, self.table_name)
        expect_df = df_date[df_date['date'] <= '2020-10-12']
        df_date['col2'] = 3
        self.sql.dump_replace(df_in = df_date , table_name_in = self.table_name, list_key = ['date'],
                             math_logic = {'date' : {'logic' : '>', 'value' : '2020-10-12', 'type' : 'date'}})
        df_read = self.sql.read(self.table_name)
        expect_df = expect_df.append(df_date)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace3(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2','col3'])
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1.append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new1_str(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = 'col1')
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,2,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new1(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,2,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new2(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2'], debug = False)
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,4,6,7,8] , 'col2' : [1,1,2,2,3,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace_null1(self) :
        df1_n = pd.DataFrame({'col1' : [1,None,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        df2_n = pd.DataFrame({'col1' : [4,5,6,7,None] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        df1_n.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')
        self.sql.dump_replace(df_in = df2_n , table_name_in = self.table_name, list_key = ['col1'] , debug = True)
        df_read = self.sql.read(self.table_name).sort_values('col1').reset_index(drop = True)
        expect_df = pd.DataFrame({'col1' : [1,3,4,5,6,7,None] , 'col2' : [1,2,3,3,4,4,5] , 'col3' : [1,1,2,2,2,2,2]})
        result, expect = help_test(df_read.fillna(99), expect_df.fillna(99))
        self.assertEqual(result, expect)

    def test_replace_null2(self) :
        df1_n = pd.DataFrame({'col1' : [1,None,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        df2_n = pd.DataFrame({'col1' : [4,5,6,7,None] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        df1_n.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')
        self.sql.dump_replace(df_in = df2_n , table_name_in = self.table_name, list_key = ['col1','col2'] , debug = True)
        df_read = self.sql.read(self.table_name).sort_values('col1').reset_index(drop = True)
        expect_df = pd.DataFrame({ 'col1' : [1,None,3,4,4,5,6,7,None] , 'col2' : [1,1,2,2,3,3,4,4,5] 
                                  ,'col3' : [1,1,1,1,2,2,2,2,2]}).sort_values('col1').reset_index(drop = True)

        result, expect = help_test(df_read.fillna(99), expect_df.fillna(99))
        self.assertEqual(result, expect)

unittest.main(argv = ['first-arg-is-ignored'], exit = False)

C:\Users\chanonkr\Anaconda3\lib\site-packages\sqlalchemy\dialects\mssql\pymssql.py:61: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  module = __import__("pymssql")


Connection OK
Start Filter Existing data from df at  2021-05-09 16:32:07.251883


.

Dump data to  unit_test_git  End  2021-05-09 16:32:07.786915
Connection OK
Start Filter Existing data from df at  2021-05-09 16:32:08.740224
Dump data to  unit_test_git  End  2021-05-09 16:32:08.918324


.

...

.

Connection OK
Drop Existing Table at  2021-05-09 16:32:23.818289


.

Dump data to  unit_test_git  End  2021-05-09 16:32:24.090450



----------------------------------------------------------------------
Ran 14 tests in 23.476s

OK


# Parallel MSSQL

In [4]:
test_mssql = setting['MSSQL']

class SQL_Test_class(unittest.TestCase):
    def setUp(self):
        self.sql = da_tran_SQL(sql_type = test_mssql['type'],
                                host_name = test_mssql['host'],
                                database_name = test_mssql['database'],
                                user = test_mssql['user'],
                                password = test_mssql['password'],
                                partition_size = 2, 
                                parallel_dump = True, 
                                max_parallel = 2)
        
        self.df1 = pd.DataFrame({'col1' : [1,2,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        self.df2 = pd.DataFrame({'col1' : [4,5,6,7,8] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        self.table_name = 'unit_test_git'
        self.df1.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')
        self.sql.parallel_dump = True
        self.sql.max_parallel = 2
        self.sql.partition_size = 2

    def test_sp(self):
        sql_q = """ CREATE OR ALTER PROCEDURE unit_test_git_SP (@PARAM1 AS VARCHAR(100))
                    AS
                    BEGIN
                    SELECT * FROM {} WHERE col1 = @PARAM1 
                    END""".format(self.table_name)
        self.sql.engine.execute(sql_q)
        df_read = self.sql.read('unit_test_git_SP', SP = True , param = {'@PARAM1' : 1})
        expect_df = self.df1[self.df1['col1'] == 1]
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_read_sql(self):
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_whole(self):
        self.sql.dump_whole(self.df2, self.table_name)
        df_read = self.sql.read(self.table_name)
        expect_df = self.df2
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    
    def test_replace1(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,3,3,4,4,5] , 'col3' : [1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace1_str(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = 'col1')
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,3,3,4,4,5] , 'col3' : [1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace2(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,4,5,6,7,8] , 'col2' : [1,1,2,2,3,3,4,4,5] , 'col3' : [1,1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace3(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2','col3'])
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1.append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace_logic(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'],
                             math_logic = {'col1' : {'logic' : '>', 'value' : 1}})
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1[self.df1['col1'] <= 1].append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace_logic_date(self) :
        df_date = pd.DataFrame({'col1' : [1,2,3,4,5], 
                                'date' : pd.date_range('2020-10-10','2020-10-14'), 'col2' :[1,1,1,1,1]})
        self.sql.dump_whole(df_date, self.table_name)
        expect_df = df_date[df_date['date'] <= '2020-10-12']
        df_date['col2'] = 3
        self.sql.dump_replace(df_in = df_date , table_name_in = self.table_name, list_key = ['date'],
                             math_logic = {'date' : {'logic' : '>', 'value' : '2020-10-12', 'type' : 'date'}},debug = True)
        df_read = self.sql.read(self.table_name)
        expect_df = expect_df.append(df_date)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace3(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2','col3'])
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1.append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new1_str(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = 'col1')
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,2,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new1(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,2,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new2(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2'], debug = False)
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,4,6,7,8] , 'col2' : [1,1,2,2,3,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace_null1(self) :
        df1_n = pd.DataFrame({'col1' : [1,None,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        df2_n = pd.DataFrame({'col1' : [4,5,6,7,None] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        df1_n.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')
        self.sql.dump_replace(df_in = df2_n , table_name_in = self.table_name, list_key = ['col1'] , debug = True)
        df_read = self.sql.read(self.table_name).sort_values('col1').reset_index(drop = True)
        expect_df = pd.DataFrame({'col1' : [1,3,4,5,6,7,None] , 'col2' : [1,2,3,3,4,4,5] , 'col3' : [1,1,2,2,2,2,2]})
        result, expect = help_test(df_read.fillna(99), expect_df.fillna(99))
        self.assertEqual(result, expect)

    def test_replace_null2(self) :
        df1_n = pd.DataFrame({'col1' : [1,None,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        df2_n = pd.DataFrame({'col1' : [4,5,6,7,None] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        df1_n.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')
        self.sql.dump_replace(df_in = df2_n , table_name_in = self.table_name, list_key = ['col1','col2'] , debug = True)
        df_read = self.sql.read(self.table_name).sort_values('col1').reset_index(drop = True)
        expect_df = pd.DataFrame({ 'col1' : [1,None,3,4,4,5,6,7,None] , 'col2' : [1,1,2,2,3,3,4,4,5] 
                                  ,'col3' : [1,1,1,1,2,2,2,2,2]}).sort_values('col1').reset_index(drop = True)

        result, expect = help_test(df_read.fillna(99), expect_df.fillna(99))
        self.assertEqual(result, expect)

unittest.main(argv = ['first-arg-is-ignored'], exit = False)

Connection OK
Start Filter Existing data from df at  2021-05-09 16:32:25.132061


.

Dump data to  unit_test_git  End  2021-05-09 16:32:25.447143
Connection OK
Start Filter Existing data from df at  2021-05-09 16:32:26.437345


.

Dump data to  unit_test_git  End  2021-05-09 16:32:26.732596
Connection OK
Start Filter Existing data from df at  2021-05-09 16:32:27.727393


...

.

Connection OK
Drop Existing Table at  2021-05-09 16:32:44.200839


.

Dump data to  unit_test_git  End  2021-05-09 16:32:44.663932



----------------------------------------------------------------------
Ran 14 tests in 20.521s

OK


## ODBC MSSQL

In [5]:
test_mssql = setting['MSSQL']

class SQL_Test_class(unittest.TestCase):
    def setUp(self):
        self.sql = da_tran_SQL(sql_type = test_mssql['type'],
                                host_name = test_mssql['host'],
                                database_name = test_mssql['database'],
                                user = test_mssql['user'],
                                password = test_mssql['password'],
                                driver = 'pyodbc', parameter = 'driver=SQL+Server')
        self.df1 = pd.DataFrame({'col1' : [1,2,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        self.df2 = pd.DataFrame({'col1' : [4,5,6,7,8] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        self.table_name = 'unit_test_git'
        self.df1.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')

    def test_sp(self):
        sql_q = """ CREATE OR ALTER PROCEDURE unit_test_git_SP (@PARAM1 AS VARCHAR(100))
                    AS
                    BEGIN
                    SELECT * FROM {} WHERE col1 = @PARAM1 
                    END""".format(self.table_name)
        self.sql.engine.execute(sql_q)
        df_read = self.sql.read('unit_test_git_SP', SP = True , param = {'@PARAM1' : 1})
        expect_df = self.df1[self.df1['col1'] == 1]
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_read_sql(self):
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_whole(self):
        self.sql.dump_whole(self.df2, self.table_name)
        df_read = self.sql.read(self.table_name)
        expect_df = self.df2
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    
    def test_replace1(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,3,3,4,4,5] , 'col3' : [1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace1_str(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = 'col1')
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,3,3,4,4,5] , 'col3' : [1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace2(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,4,5,6,7,8] , 'col2' : [1,1,2,2,3,3,4,4,5] , 'col3' : [1,1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace3(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2','col3'])
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1.append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace_logic(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'],
                             math_logic = {'col1' : {'logic' : '>', 'value' : 1}})
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1[self.df1['col1'] <= 1].append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace_logic_date(self) :
        df_date = pd.DataFrame({'col1' : [1,2,3,4,5], 
                                'date' : pd.date_range('2020-10-10','2020-10-14'), 'col2' :[1,1,1,1,1]})
        
        self.sql.dump_whole(df_date, self.table_name)
        expect_df = df_date[df_date['date'] <= '2020-10-12']
        df_date['col2'] = 3
        self.sql.dump_replace(df_in = df_date , table_name_in = self.table_name, list_key = ['date'],
                             math_logic = {'date' : {'logic' : '>', 'value' : '2020-10-12', 'type' : 'date'}})
        df_read = self.sql.read(self.table_name)
        expect_df = expect_df.append(df_date)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace3(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2','col3'])
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1.append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new1_str(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = 'col1')
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,2,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new1(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,2,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new2(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2'], debug = False)
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,4,6,7,8] , 'col2' : [1,1,2,2,3,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace_null1(self) :
        df1_n = pd.DataFrame({'col1' : [1,None,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        df2_n = pd.DataFrame({'col1' : [4,5,6,7,None] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        df1_n.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')
        self.sql.dump_replace(df_in = df2_n , table_name_in = self.table_name, list_key = ['col1'] , debug = True)
        df_read = self.sql.read(self.table_name).sort_values('col1').reset_index(drop = True)
        expect_df = pd.DataFrame({'col1' : [1,3,4,5,6,7,None] , 'col2' : [1,2,3,3,4,4,5] , 'col3' : [1,1,2,2,2,2,2]})
        result, expect = help_test(df_read.fillna(99), expect_df.fillna(99))
        self.assertEqual(result, expect)

    def test_replace_null2(self) :
        df1_n = pd.DataFrame({'col1' : [1,None,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        df2_n = pd.DataFrame({'col1' : [4,5,6,7,None] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        df1_n.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')
        self.sql.dump_replace(df_in = df2_n , table_name_in = self.table_name, list_key = ['col1','col2'] , debug = True)
        df_read = self.sql.read(self.table_name).sort_values('col1').reset_index(drop = True)
        expect_df = pd.DataFrame({ 'col1' : [1,None,3,4,4,5,6,7,None] , 'col2' : [1,1,2,2,3,3,4,4,5] 
                                  ,'col3' : [1,1,1,1,2,2,2,2,2]}).sort_values('col1').reset_index(drop = True)

        result, expect = help_test(df_read.fillna(99), expect_df.fillna(99))
        self.assertEqual(result, expect)

unittest.main(argv = ['first-arg-is-ignored'], exit = False)

Connection OK


.

Start Filter Existing data from df at  2021-05-09 16:32:48.442719
Dump data to  unit_test_git  End  2021-05-09 16:32:48.569990
Connection OK


.

Start Filter Existing data from df at  2021-05-09 16:32:49.389020
Dump data to  unit_test_git  End  2021-05-09 16:32:49.514683
Connection OK


...

.

Connection OK


.

Drop Existing Table at  2021-05-09 16:33:01.111973
Dump data to  unit_test_git  End  2021-05-09 16:33:01.268957



----------------------------------------------------------------------
Ran 14 tests in 16.535s

OK


# MYSQL
## Default MYSQL

In [6]:
test_mysql = setting['MYSQL']
    
class SQL_Test_class(unittest.TestCase):
    def setUp(self):
        self.sql = da_tran_SQL(sql_type = test_mysql['type'],
                                host_name = test_mysql['host'],
                                database_name = test_mysql['database'],
                                user = test_mysql['user'],
                                password = test_mysql['password'])
        self.df1 = pd.DataFrame({'col1' : [1,2,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        self.df2 = pd.DataFrame({'col1' : [4,5,6,7,8] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        self.table_name = 'unit_test_git'
        self.df1.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')

    def test_sp(self):
        sql_q = """ drop procedure if exists unit_test_git_SP;"""
        self.sql.engine.execute(sql_q)
        sql_q = """ CREATE PROCEDURE unit_test_git_SP
                    (PARAM1 VARCHAR(100))
                    BEGIN
                    SELECT * FROM {} WHERE col1 = PARAM1 ;
                    end;""".format(self.table_name)
        self.sql.engine.execute(sql_q)
        df_read = self.sql.read('unit_test_git_SP', SP = True , param = {'PARAM1' : 1})
        expect_df = self.df1[self.df1['col1'] == 1]
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)


    def test_read_sql(self):
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_whole(self):
        self.sql.dump_whole(self.df2, self.table_name)
        df_read = self.sql.read(self.table_name)
        expect_df = self.df2
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace1(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,3,3,4,4,5] , 'col3' : [1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace1_str(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = 'col1')
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,3,3,4,4,5] , 'col3' : [1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace2(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,4,5,6,7,8] , 'col2' : [1,1,2,2,3,3,4,4,5] , 'col3' : [1,1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace3(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2','col3'])
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1.append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace_logic(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'],
                             math_logic = {'col1' : {'logic' : '>', 'value' : 1}})
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1[self.df1['col1'] <= 1].append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace_logic_date(self) :
        df_date = pd.DataFrame({'col1' : [1,2,3,4,5], 
                                'date' : pd.date_range('2020-10-10','2020-10-14'), 'col2' :[1,1,1,1,1]})
        self.sql.dump_whole(df_date, self.table_name)
        expect_df = df_date[df_date['date'] <= '2020-10-12']
        df_date['col2'] = 3
        self.sql.dump_replace(df_in = df_date , table_name_in = self.table_name, list_key = ['date'],
                             math_logic = {'date' : {'logic' : '>', 'value' : '2020-10-12', 'type' : 'date'}})
        df_read = self.sql.read(self.table_name)
        expect_df = expect_df.append(df_date)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace3(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2','col3'])
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1.append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new1_str(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = 'col1')
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,2,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new1(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,2,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new2(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2'], debug = False)
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,4,6,7,8] , 'col2' : [1,1,2,2,3,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace_null1(self) :
        df1_n = pd.DataFrame({'col1' : [1,None,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        df2_n = pd.DataFrame({'col1' : [4,5,6,7,None] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        df1_n.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')
        self.sql.dump_replace(df_in = df2_n , table_name_in = self.table_name, list_key = ['col1'] , debug = True)
        df_read = self.sql.read(self.table_name).sort_values('col1').reset_index(drop = True)
        expect_df = pd.DataFrame({'col1' : [1,3,4,5,6,7,None] , 'col2' : [1,2,3,3,4,4,5] , 'col3' : [1,1,2,2,2,2,2]})
        result, expect = help_test(df_read.fillna(99), expect_df.fillna(99))
        self.assertEqual(result, expect)

    def test_replace_null2(self) :
        df1_n = pd.DataFrame({'col1' : [1,None,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        df2_n = pd.DataFrame({'col1' : [4,5,6,7,None] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        df1_n.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')
        self.sql.dump_replace(df_in = df2_n , table_name_in = self.table_name, list_key = ['col1','col2'] , debug = True)
        df_read = self.sql.read(self.table_name).sort_values('col1').reset_index(drop = True)
        expect_df = pd.DataFrame({ 'col1' : [1,None,3,4,4,5,6,7,None] , 'col2' : [1,1,2,2,3,3,4,4,5] 
                                  ,'col3' : [1,1,1,1,2,2,2,2,2]}).sort_values('col1').reset_index(drop = True)

        result, expect = help_test(df_read.fillna(99), expect_df.fillna(99))
        self.assertEqual(result, expect)

unittest.main(argv = ['first-arg-is-ignored'], exit = False)

Connection OK
Start Filter Existing data from df at  2021-05-09 16:33:02.599914
Dump data to  unit_test_git  End  2021-05-09 16:33:02.798593


.

Connection OK
Start Filter Existing data from df at  2021-05-09 16:33:03.903249
Dump data to  unit_test_git  End  2021-05-09 16:33:04.101085


.

Connection OK
Start Filter Existing data from df at  2021-05-09 16:33:05.057359


...

.

Connection OK
Drop Existing Table at  2021-05-09 16:33:20.156343


.

Dump data to  unit_test_git  End  2021-05-09 16:33:20.502102



----------------------------------------------------------------------
Ran 14 tests in 19.155s

OK


## Parallel MYSQL

In [7]:
test_mysql = setting['MYSQL']
    
class SQL_Test_class(unittest.TestCase):
    def setUp(self):
        self.sql = da_tran_SQL(sql_type = test_mysql['type'],
                                host_name = test_mysql['host'],
                                database_name = test_mysql['database'],
                                user = test_mysql['user'],
                                password = test_mysql['password'],
                                partition_size = 2, 
                                parallel_dump = True, 
                                max_parallel = 2)
        
        self.df1 = pd.DataFrame({'col1' : [1,2,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        self.df2 = pd.DataFrame({'col1' : [4,5,6,7,8] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        self.table_name = 'unit_test_git'
        self.df1.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')

    def test_sp(self):
        sql_q = """ drop procedure if exists unit_test_git_SP;"""
        self.sql.engine.execute(sql_q)
        sql_q = """ CREATE PROCEDURE unit_test_git_SP
                    (PARAM1 VARCHAR(100))
                    BEGIN
                    SELECT * FROM {} WHERE col1 = PARAM1 ;
                    end;""".format(self.table_name)
        self.sql.engine.execute(sql_q)
        df_read = self.sql.read('unit_test_git_SP', SP = True , param = {'PARAM1' : 1})
        expect_df = self.df1[self.df1['col1'] == 1]
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)


    def test_read_sql(self):
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_whole(self):
        self.sql.dump_whole(self.df2, self.table_name)
        df_read = self.sql.read(self.table_name)
        expect_df = self.df2
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace1(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,3,3,4,4,5] , 'col3' : [1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace1_str(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = 'col1')
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,3,3,4,4,5] , 'col3' : [1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace2(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,4,5,6,7,8] , 'col2' : [1,1,2,2,3,3,4,4,5] , 'col3' : [1,1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace3(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2','col3'])
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1.append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace_logic(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'],
                             math_logic = {'col1' : {'logic' : '>', 'value' : 1}})
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1[self.df1['col1'] <= 1].append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace_logic_date(self) :
        df_date = pd.DataFrame({'col1' : [1,2,3,4,5], 
                                'date' : pd.date_range('2020-10-10','2020-10-14'), 'col2' :[1,1,1,1,1]})
        self.sql.dump_whole(df_date, self.table_name)
        expect_df = df_date[df_date['date'] <= '2020-10-12']
        df_date['col2'] = 3
        self.sql.dump_replace(df_in = df_date , table_name_in = self.table_name, list_key = ['date'],
                             math_logic = {'date' : {'logic' : '>', 'value' : '2020-10-12', 'type' : 'date'}})
        df_read = self.sql.read(self.table_name)
        expect_df = expect_df.append(df_date)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace3(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2','col3'])
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1.append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new1_str(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = 'col1')
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,2,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new1(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,2,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new2(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2'], debug = False)
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,4,6,7,8] , 'col2' : [1,1,2,2,3,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace_null1(self) :
        df1_n = pd.DataFrame({'col1' : [1,None,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        df2_n = pd.DataFrame({'col1' : [4,5,6,7,None] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        df1_n.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')
        self.sql.dump_replace(df_in = df2_n , table_name_in = self.table_name, list_key = ['col1'] , debug = True)
        df_read = self.sql.read(self.table_name).sort_values('col1').reset_index(drop = True)
        expect_df = pd.DataFrame({'col1' : [1,3,4,5,6,7,None] , 'col2' : [1,2,3,3,4,4,5] , 'col3' : [1,1,2,2,2,2,2]})
        result, expect = help_test(df_read.fillna(99), expect_df.fillna(99))
        self.assertEqual(result, expect)

    def test_replace_null2(self) :
        df1_n = pd.DataFrame({'col1' : [1,None,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        df2_n = pd.DataFrame({'col1' : [4,5,6,7,None] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        df1_n.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')
        self.sql.dump_replace(df_in = df2_n , table_name_in = self.table_name, list_key = ['col1','col2'] , debug = True)
        df_read = self.sql.read(self.table_name).sort_values('col1').reset_index(drop = True)
        expect_df = pd.DataFrame({ 'col1' : [1,None,3,4,4,5,6,7,None] , 'col2' : [1,1,2,2,3,3,4,4,5] 
                                  ,'col3' : [1,1,1,1,2,2,2,2,2]}).sort_values('col1').reset_index(drop = True)

        result, expect = help_test(df_read.fillna(99), expect_df.fillna(99))
        self.assertEqual(result, expect)

unittest.main(argv = ['first-arg-is-ignored'], exit = False)

Connection OK
Start Filter Existing data from df at  2021-05-09 16:33:21.683011


.

Dump data to  unit_test_git  End  2021-05-09 16:33:22.002360
Connection OK
Start Filter Existing data from df at  2021-05-09 16:33:23.044502


.

Dump data to  unit_test_git  End  2021-05-09 16:33:23.362046
Connection OK
Start Filter Existing data from df at  2021-05-09 16:33:24.418409


...

.

Connection OK
Drop Existing Table at  2021-05-09 16:33:40.491269


.

Dump data to  unit_test_git  End  2021-05-09 16:33:41.031152



----------------------------------------------------------------------
Ran 14 tests in 20.482s

OK


# PostgreSQL
## Default PostgreSQL

In [8]:
test_postgresql = setting['POSTGRESQL']

class SQL_Test_class(unittest.TestCase):
    def setUp(self):
        self.sql = da_tran_SQL(sql_type = test_postgresql['type'],
                                host_name = test_postgresql['host'],
                                database_name = test_postgresql['database'],
                                user = test_postgresql['user'],
                                password = test_postgresql['password'])
        self.df1 = pd.DataFrame({'col1' : [1,2,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        self.df2 = pd.DataFrame({'col1' : [4,5,6,7,8] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        self.table_name = 'unit_test_git'
        self.df1.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')

#     def test_sp(self):
#         sql_q = """ CREATE OR REPLACE PROCEDURE unit_test_git_SP (PARAM1 INT)
#                     RETURNS TABLE (col1 int, col2 int, col3 int)
#                     language plpgsql
#                     AS $$
#                     BEGIN
#                     RETURN QUERY(SELECT * FROM {} WHERE col1 = PARAM1) ;
#                     END ; $$""".format(self.table_name)
#         self.sql.engine.execute(sql_q)
#         df_read = self.sql.read('unit_test_git_SP', SP = True , param = {'PARAM1' : 1})
#         expect_df = self.df1[self.df1['col1'] == 1]
#         result, expect = help_test(df_read, expect_df)
#         self.assertEqual(result, expect)
        
    def test_read_sql(self):
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_whole(self):
        self.sql.dump_whole(self.df2, self.table_name)
        df_read = self.sql.read(self.table_name)
        expect_df = self.df2
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace1(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,3,3,4,4,5] , 'col3' : [1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace1_str(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = 'col1')
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,3,3,4,4,5] , 'col3' : [1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace2(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,4,5,6,7,8] , 'col2' : [1,1,2,2,3,3,4,4,5] , 'col3' : [1,1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace3(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2','col3'])
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1.append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace_logic(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'],
                             math_logic = {'col1' : {'logic' : '>', 'value' : 1}})
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1[self.df1['col1'] <= 1].append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace_logic_date(self) :
        df_date = pd.DataFrame({'col1' : [1,2,3,4,5], 
                                'date' : pd.date_range('2020-10-10','2020-10-14'), 'col2' :[1,1,1,1,1]})
        
        self.sql.dump_whole(df_date, self.table_name)
        expect_df = df_date[df_date['date'] <= '2020-10-12']
        df_date['col2'] = 3
        self.sql.dump_replace(df_in = df_date , table_name_in = self.table_name, list_key = ['date'],
                             math_logic = {'date' : {'logic' : '>', 'value' : '2020-10-12', 'type' : 'date'}})
        df_read = self.sql.read(self.table_name)
        expect_df = expect_df.append(df_date)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace3(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2','col3'])
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1.append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new1_str(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = 'col1')
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,2,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new1(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,2,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new2(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2'], debug = False)
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,4,6,7,8] , 'col2' : [1,1,2,2,3,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace_null1(self) :
        df1_n = pd.DataFrame({'col1' : [1,None,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        df2_n = pd.DataFrame({'col1' : [4,5,6,7,None] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        df1_n.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')
        self.sql.dump_replace(df_in = df2_n , table_name_in = self.table_name, list_key = ['col1'] , debug = True)
        df_read = self.sql.read(self.table_name).sort_values('col1').reset_index(drop = True)
        expect_df = pd.DataFrame({'col1' : [1,3,4,5,6,7,None] , 'col2' : [1,2,3,3,4,4,5] , 'col3' : [1,1,2,2,2,2,2]})
        result, expect = help_test(df_read.fillna(99), expect_df.fillna(99))
        self.assertEqual(result, expect)

    def test_replace_null2(self) :
        df1_n = pd.DataFrame({'col1' : [1,None,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        df2_n = pd.DataFrame({'col1' : [4,5,6,7,None] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        df1_n.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')
        self.sql.dump_replace(df_in = df2_n , table_name_in = self.table_name, list_key = ['col1','col2'] , debug = True)
        df_read = self.sql.read(self.table_name).sort_values('col1').reset_index(drop = True)
        expect_df = pd.DataFrame({ 'col1' : [1,None,3,4,4,5,6,7,None] , 'col2' : [1,1,2,2,3,3,4,4,5] 
                                  ,'col3' : [1,1,1,1,2,2,2,2,2]}).sort_values('col1').reset_index(drop = True)

        result, expect = help_test(df_read.fillna(99), expect_df.fillna(99))
        self.assertEqual(result, expect)
        
unittest.main(argv = ['first-arg-is-ignored'], exit = False)

Connection OK
Start Filter Existing data from df at  2021-05-09 16:34:00.316341
Dump data to  unit_test_git  End  2021-05-09 16:34:02.900893


.

Connection OK
Start Filter Existing data from df at  2021-05-09 16:34:19.443975
Dump data to  unit_test_git  End  2021-05-09 16:34:21.962341


.

Connection OK
Start Filter Existing data from df at  2021-05-09 16:34:38.595584
Dump data to  unit_test_git  End  2021-05-09 16:34:41.130989


...

.

Connection OK
Start delete old data at 2021-05-09 16:37:40.565335
Delete Last ['col1', 'col2'] at 2021-05-09 16:37:41.401204
DELETE FROM "unit_test_git" where ("col1" in ('4.0', '5.0', '6.0', '7.0') OR "col1" IS NULL) and "col2" in ('3', '4', '5')
Dump data to  unit_test_git  End  2021-05-09 16:37:43.070481


.

Connection OK
Drop Existing Table at  2021-05-09 16:38:00.348459
Dump data to  unit_test_git  End  2021-05-09 16:38:03.964938


.
----------------------------------------------------------------------
Ran 13 tests in 263.309s

OK


## Parallel PostgreSQL

In [9]:
test_postgresql = setting['POSTGRESQL']

class SQL_Test_class(unittest.TestCase):
    def setUp(self):
        self.sql = da_tran_SQL(sql_type = test_postgresql['type'],
                                host_name = test_postgresql['host'],
                                database_name = test_postgresql['database'],
                                user = test_postgresql['user'],
                                password = test_postgresql['password'],
                                partition_size = 2, 
                                parallel_dump = True, 
                                max_parallel = 2)
        
        self.df1 = pd.DataFrame({'col1' : [1,2,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        self.df2 = pd.DataFrame({'col1' : [4,5,6,7,8] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        self.table_name = 'unit_test_git'
        self.df1.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')

#     def test_sp(self):
#         sql_q = """ CREATE OR REPLACE PROCEDURE unit_test_git_SP (PARAM1 INT)
#                     RETURNS TABLE (col1 int, col2 int, col3 int)
#                     language plpgsql
#                     AS $$
#                     BEGIN
#                     RETURN QUERY(SELECT * FROM {} WHERE col1 = PARAM1) ;
#                     END ; $$""".format(self.table_name)
#         self.sql.engine.execute(sql_q)
#         df_read = self.sql.read('unit_test_git_SP', SP = True , param = {'PARAM1' : 1})
#         expect_df = self.df1[self.df1['col1'] == 1]
#         result, expect = help_test(df_read, expect_df)
#         self.assertEqual(result, expect)
        
    def test_read_sql(self):
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_whole(self):
        self.sql.dump_whole(self.df2, self.table_name)
        df_read = self.sql.read(self.table_name)
        expect_df = self.df2
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace1(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,3,3,4,4,5] , 'col3' : [1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace1_str(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = 'col1')
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,3,3,4,4,5] , 'col3' : [1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace2(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,4,5,6,7,8] , 'col2' : [1,1,2,2,3,3,4,4,5] , 'col3' : [1,1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace3(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2','col3'])
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1.append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace_logic(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'],
                             math_logic = {'col1' : {'logic' : '>', 'value' : 1}})
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1[self.df1['col1'] <= 1].append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace_logic_date(self) :
        df_date = pd.DataFrame({'col1' : [1,2,3,4,5], 
                                'date' : pd.date_range('2020-10-10','2020-10-14'), 'col2' :[1,1,1,1,1]})
        
        self.sql.dump_whole(df_date, self.table_name)
        expect_df = df_date[df_date['date'] <= '2020-10-12']
        df_date['col2'] = 3
        self.sql.dump_replace(df_in = df_date , table_name_in = self.table_name, list_key = ['date'],
                             math_logic = {'date' : {'logic' : '>', 'value' : '2020-10-12', 'type' : 'date'}})
        df_read = self.sql.read(self.table_name)
        expect_df = expect_df.append(df_date)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace3(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2','col3'])
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1.append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new1_str(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = 'col1')
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,2,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new1(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,2,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new2(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2'], debug = False)
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,4,6,7,8] , 'col2' : [1,1,2,2,3,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace_null1(self) :
        df1_n = pd.DataFrame({'col1' : [1,None,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        df2_n = pd.DataFrame({'col1' : [4,5,6,7,None] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        df1_n.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')
        self.sql.dump_replace(df_in = df2_n , table_name_in = self.table_name, list_key = ['col1'] , debug = True)
        df_read = self.sql.read(self.table_name).sort_values('col1').reset_index(drop = True)
        expect_df = pd.DataFrame({'col1' : [1,3,4,5,6,7,None] , 'col2' : [1,2,3,3,4,4,5] , 'col3' : [1,1,2,2,2,2,2]})
        result, expect = help_test(df_read.fillna(99), expect_df.fillna(99))
        self.assertEqual(result, expect)

    def test_replace_null2(self) :
        df1_n = pd.DataFrame({'col1' : [1,None,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        df2_n = pd.DataFrame({'col1' : [4,5,6,7,None] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        df1_n.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')
        self.sql.dump_replace(df_in = df2_n , table_name_in = self.table_name, list_key = ['col1','col2'] , debug = True)
        df_read = self.sql.read(self.table_name).sort_values('col1').reset_index(drop = True)
        expect_df = pd.DataFrame({ 'col1' : [1,None,3,4,4,5,6,7,None] , 'col2' : [1,1,2,2,3,3,4,4,5] 
                                  ,'col3' : [1,1,1,1,2,2,2,2,2]}).sort_values('col1').reset_index(drop = True)

        result, expect = help_test(df_read.fillna(99), expect_df.fillna(99))
        self.assertEqual(result, expect)
        
unittest.main(argv = ['first-arg-is-ignored'], exit = False)

Connection OK
Start Filter Existing data from df at  2021-05-09 16:38:20.960896
Dump data to  unit_test_git  End  2021-05-09 16:38:25.376006


.

Connection OK
Start Filter Existing data from df at  2021-05-09 16:38:42.627041
Dump data to  unit_test_git  End  2021-05-09 16:38:47.074026


.

Connection OK
Start Filter Existing data from df at  2021-05-09 16:39:03.665870
Dump data to  unit_test_git  End  2021-05-09 16:39:07.909443


...

.

Connection OK
Start delete old data at 2021-05-09 16:42:37.774360
Delete Last ['col1', 'col2'] at 2021-05-09 16:42:38.630517
DELETE FROM "unit_test_git" where ("col1" in ('4.0', '5.0', '6.0', '7.0') OR "col1" IS NULL) and "col2" in ('3', '4', '5')
Dump data to  unit_test_git  End  2021-05-09 16:42:43.708439


.

Connection OK
Drop Existing Table at  2021-05-09 16:42:58.783113
Dump data to  unit_test_git  End  2021-05-09 16:43:04.373376


.
----------------------------------------------------------------------
Ran 13 tests in 300.162s

OK


# SQLite
## Default SQLite : No Datetime in SQLite

In [10]:
class SQL_Test_class(unittest.TestCase):
    def setUp(self):
        self.sql = da_tran_SQL(sql_type = 'sqlite', 
                    host_name = 'test.db', 
                    database_name = '', 
                    user = '', 
                    password = '' )
        self.df1 = pd.DataFrame({'col1' : [1,2,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        self.df2 = pd.DataFrame({'col1' : [4,5,6,7,8] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        self.table_name = 'unit_test_git'
        self.df1.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')
        
    def test_read_sql(self):
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_whole(self):
        self.sql.dump_whole(self.df2, self.table_name)
        df_read = self.sql.read(self.table_name)
        expect_df = self.df2
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    
    def test_replace1(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,3,3,4,4,5] , 'col3' : [1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace1_str(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = 'col1')
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,3,3,4,4,5] , 'col3' : [1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace2(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,4,5,6,7,8] , 'col2' : [1,1,2,2,3,3,4,4,5] , 'col3' : [1,1,1,1,2,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace3(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2','col3'])
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1.append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_replace_logic(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'],
                             math_logic = {'col1' : {'logic' : '>', 'value' : 1}})
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1[self.df1['col1'] <= 1].append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace3(self) :
        self.sql.dump_replace(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2','col3'])
        df_read = self.sql.read(self.table_name)
        expect_df = self.df1.append(self.df2)
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new1_str(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = 'col1')
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,2,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new1(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1'])
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,6,7,8] , 'col2' : [1,1,2,2,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)

    def test_new2(self) :
        self.sql.dump_new(df_in = self.df2 , table_name_in = self.table_name, list_key = ['col1','col2'], debug = False)
        df_read = self.sql.read(self.table_name)
        expect_df = pd.DataFrame({'col1' : [1,2,3,4,5,4,6,7,8] , 'col2' : [1,1,2,2,3,3,4,4,5] , 'col3' : [1,1,1,1,1,2,2,2,2]})
        result, expect = help_test(df_read, expect_df)
        self.assertEqual(result, expect)
        
    def test_replace_null1(self) :
        df1_n = pd.DataFrame({'col1' : [1,None,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        df2_n = pd.DataFrame({'col1' : [4,5,6,7,None] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        df1_n.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')
        self.sql.dump_replace(df_in = df2_n , table_name_in = self.table_name, list_key = ['col1'] , debug = True)
        df_read = self.sql.read(self.table_name).sort_values('col1').reset_index(drop = True)
        expect_df = pd.DataFrame({'col1' : [1,3,4,5,6,7,None] , 'col2' : [1,2,3,3,4,4,5] , 'col3' : [1,1,2,2,2,2,2]})
        result, expect = help_test(df_read.fillna(99), expect_df.fillna(99))
        self.assertEqual(result, expect)

    def test_replace_null2(self) :
        df1_n = pd.DataFrame({'col1' : [1,None,3,4,5] , 'col2' : [1,1,2,2,3] , 'col3' : [1,1,1,1,1]})
        df2_n = pd.DataFrame({'col1' : [4,5,6,7,None] , 'col2' : [3,3,4,4,5] , 'col3' : [2,2,2,2,2]})
        df1_n.to_sql(self.table_name, con = self.sql.engine, index = False, if_exists = 'replace')
        self.sql.dump_replace(df_in = df2_n , table_name_in = self.table_name, list_key = ['col1','col2'] , debug = True)
        df_read = self.sql.read(self.table_name).sort_values('col1').reset_index(drop = True)
        expect_df = pd.DataFrame({ 'col1' : [1,None,3,4,4,5,6,7,None] , 'col2' : [1,1,2,2,3,3,4,4,5] 
                                  ,'col3' : [1,1,1,1,2,2,2,2,2]}).sort_values('col1').reset_index(drop = True)

        result, expect = help_test(df_read.fillna(99), expect_df.fillna(99))
        self.assertEqual(result, expect)

unittest.main(argv = ['first-arg-is-ignored'], exit = False)

.....

Connection OK
Start Filter Existing data from df at  2021-05-09 16:43:05.334766
Dump data to  unit_test_git  End  2021-05-09 16:43:05.346696
Connection OK
Start Filter Existing data from df at  2021-05-09 16:43:05.377641
Dump data to  unit_test_git  End  2021-05-09 16:43:05.387585
Connection OK
Start Filter Existing data from df at  2021-05-09 16:43:05.418539
Dump data to  unit_test_git  End  2021-05-09 16:43:05.430507
Connection OK
Connection OK
Start delete old data at 2021-05-09 16:43:05.481335
Delete Last ['col1'] at 2021-05-09 16:43:05.485324
Dump data to  unit_test_git  End  2021-05-09 16:43:05.491309
Connection OK


.....

Start delete old data at 2021-05-09 16:43:05.517246
Delete Last col1 at 2021-05-09 16:43:05.521117
Dump data to  unit_test_git  End  2021-05-09 16:43:05.528115
Connection OK
Start delete old data at 2021-05-09 16:43:05.559033
Delete Last ['col1', 'col2'] at 2021-05-09 16:43:05.564035
Dump data to  unit_test_git  End  2021-05-09 16:43:05.569977
Connection OK
Start delete old data at 2021-05-09 16:43:05.594865
Delete Last ['col1', 'col2', 'col3'] at 2021-05-09 16:43:05.595862
Dump data to  unit_test_git  End  2021-05-09 16:43:05.601885
Connection OK
Start delete old data at 2021-05-09 16:43:05.628128
Delete Last ['col1'] at 2021-05-09 16:43:05.632117
Dump data to  unit_test_git  End  2021-05-09 16:43:05.638145
Connection OK
Start delete old data at 2021-05-09 16:43:05.681024
Delete Last ['col1'] at 2021-05-09 16:43:05.685032
DELETE FROM [unit_test_git] where ([col1] in ('4.0', '5.0', '6.0', '7.0') OR [col1] IS NULL)
Dump data to  unit_test_git  End  2021-05-09 16:43:05.690992
Connection 

..

Start delete old data at 2021-05-09 16:43:05.737029
Delete Last ['col1', 'col2'] at 2021-05-09 16:43:05.740978
DELETE FROM [unit_test_git] where ([col1] in ('4.0', '5.0', '6.0', '7.0') OR [col1] IS NULL) and [col2] in ('3', '4', '5')
Dump data to  unit_test_git  End  2021-05-09 16:43:05.746975
Connection OK
Drop Existing Table at  2021-05-09 16:43:05.773248
Dump data to  unit_test_git  End  2021-05-09 16:43:05.786018



----------------------------------------------------------------------
Ran 12 tests in 0.538s

OK
